In [36]:
from flask import Flask, request
from flask_cors import CORS, cross_origin
from flask_restful import Resource, Api
from json import dumps
from flask_jsonpify import jsonify
import json
import pymongo
import urllib
from collections import Counter
import requests
from flask_swagger_ui import get_swaggerui_blueprint

In [37]:
surveyTemplate = {
        'userName' : "Hemanth",
        'surveyName' : "test",
        'surveyForm':{"0":{
            "question":"qsn1",
            "questionType":"radio",
            "options":"sd,sdf,sv",
            "rangeFrom" :"1",
            "rangeFrom" :"20"
        },
                "1":{
            "qsn":"qsn2",
            "type":"texxt",
            "options":["a","c"]
        }
            
        }
    }

surveyEntr11y= {
        'userName' : "Hemanth",
        'surveyName' : "test",
        'entryForm':{"0":"ans1",
        "1":"ans1"   
        }
    }

In [38]:
def checkSurveyExists(surveyName):
    db=client.survey
    form=db.SurveyForms.find_one({'surveyName': surveyName})
    if(form):
        return True
    return False


def mergeSingleSurveyEntry(surveyTemplate,surveyEntry):
    form=surveyTemplate["surveyForm"]
    for key in form:
        form[key]["answer"]=surveyEntry["entryForm"][key]
    return form


def mergeSurveyEntry(surveyTemplate,surveyEntry):
    form=surveyTemplate
    for key in form["surveyForm"]:
        form["surveyForm"][key]["answers"]=[]
        
    form["users"]=[]
    for entry in surveyEntry:
        form["users"].append(entry["userName"])
        for key in form["surveyForm"]:
            form["surveyForm"][key]["answers"].append(entry["entryForm"][key])
            
    for key in form["surveyForm"]:
        form["surveyForm"][key]["answers"]=Counter(form["surveyForm"][key]["answers"])
        
    return form



def verifyAccessToken(access_token):
    URL="http://127.0.0.1:5001/authorize"
    response=requests.post(url = URL,headers={'Authorization': "access_token"})
    if(response.status_code==200):
        return True
    else:
        return False
    
def getStats(surveyEntries):
    stats={}
    for i in surveyEntries[0]["entryForm"].keys():
        stats[i]={"total":{},"Male":{},"Female":{}}

    for entry in surveyEntries:
        gender=entry["gender"]
        for key in entry["entryForm"].keys():
            val=entry["entryForm"][key]
            stats[key]["total"][val]=0
            stats[key][gender][val]=0

    for entry in surveyEntries:
        gender=entry["gender"]
        for key in entry["entryForm"].keys():
            val=entry["entryForm"][key]
            stats[key]["total"][val]+=1
            stats[key][gender][val]+=1
            
    return stats

In [ ]:
app = Flask(__name__)
api = Api(app)

CORS(app)
client = pymongo.MongoClient("mongodb+srv://hemanth:"+ urllib.quote("Hemanth@123")+"@cluster0-59obx.mongodb.net/test?retryWrites=true&w=majority")



### swagger specific ###
SWAGGER_URL = '/swagger'
API_URL = '/static/swagger.json'
SWAGGERUI_BLUEPRINT = get_swaggerui_blueprint(
    SWAGGER_URL,
    API_URL,
    config={
        'app_name': "Seans-Python-Flask-REST-Boilerplate"
    }
)
app.register_blueprint(SWAGGERUI_BLUEPRINT, url_prefix=SWAGGER_URL)  


@app.route("/")
def hello():
    return jsonify({'text':'Hello World!'})


@app.route("/createSurvey",methods=['POST'])
def createSurvey(): 
    
    verifyAccessToken(request.headers["Authorization"])
    if not(verifyAccessToken):
        return json.dumps({
              "error": "access_denied"
            }), 401
    
    db=client.survey
    params=request.json
    if(checkSurveyExists(params["surveyName"])):
        return jsonify({'response':'SurveyName already Exists'}),400
    
    surveyDetails={}
    surveyDetails["userName"]=params["userName"]
    surveyDetails["surveyName"]=params["surveyName"]
    surveyDetails["surveyDescription"]=params["surveyDescription"]
    surveyDetails["surveyForm"]=params["surveyForm"]

    
    result=db.SurveyForms.insert_one(surveyDetails)
    return  jsonify({'response':'Survey successfully created'})


@app.route("/getSurvey",methods=['POST'])
def surveyForm():    
    db=client.survey
    params=request.json

    if not(checkSurveyExists(params["surveyName"])):
        return jsonify({'response':'SurveyName doesnt Exists'})
    
    result=db.SurveyForms.find_one({'surveyName': params["surveyName"]})
    del result['_id']
    return jsonify(result)


@app.route("/fillSurvey",methods=['POST'])
def surveyEntry():
    db=client.survey
    params=request.json
    if not(checkSurveyExists(params["surveyName"])):
        return jsonify({'response':'SurveyName doesnt Exist'})
    
    surveyDetails={}
    surveyDetails["userName"]=params["userName"]
    surveyDetails["surveyName"]=params["surveyName"]
    surveyDetails["entryForm"]=params["entryForm"]
    surveyDetails["gender"]=params["gender"]
    
    if(db.SurveyEntries.find_one({'surveyName': params["surveyName"],'userName':params["userName"]})):
        return  jsonify({'response':'User has already filled the survey'}),400
    
    db.SurveyEntries.insert_one(surveyDetails)
    return  jsonify({'response':'Survey successfully filled'})


@app.route("/getAllSurveys",methods=['GET'])
def getAllServeys():
    db=client.survey
    surveys=[]
    for i in db.SurveyForms.find():
        del i['_id']
        surveys.append(i)
   
    return jsonify({"results":surveys})

@app.route("/getUserSurveys",methods=['POST'])
def getUserServeys():
    params=request.json
    db=client.survey
    surveys=[]
    for i in db.SurveyForms.find({'userName': params["userName"]}):
        del i['_id']
        surveys.append(i)
   
    return jsonify({"results":surveys})

@app.route("/getSurveyEntries",methods=['POST'])
def getSurveyEntries():
    db=client.survey
    params=request.json
    print(params)
    if not(checkSurveyExists(params["surveyName"])):
        return jsonify({'response':'SurveyName doesnt Exists'})
       
    surveyEntries=[]
    for i in db.SurveyEntries.find({'surveyName': params["surveyName"]}):
        del i['_id']
        surveyEntries.append(i)
        
    stats=getStats(surveyEntries)
    return jsonify({"results":surveyEntries,"stats":stats})


@app.route("/surveystat")
def getByUserSurvey():
    db=client.SurveyEntries
    params=request.json
    print(params)
    if not(checkSurveyExists(params["surveyName"])):
        return jsonify({'response':'SurveyName doesnt Exists'})
    
    surveyEntry=db.SurveyEntries.find({'surveyName': params["surveyName"],'userName':params["userName"]})
    surveyTemplate=db.SurveyForms.find({'surveyName': params["surveyName"]})
    
    return mergeSurventry(surveyTemplate,surveyEntry)

if __name__ == '__main__':
   app.run(port=5002)



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5002/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Oct/2019 16:24:56] "OPTIONS /getSurvey HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 16:24:56] "OPTIONS /getAllSurveys HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 16:24:59] "GET /getAllSurveys HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 16:24:59] "POST /getSurvey HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 16:25:23] "OPTIONS /getSurvey HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 16:25:23] "OPTIONS /getAllSurveys HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 16:25:23] "POST /getSurvey HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 16:25:23] "GET /getAllSurveys HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 16:25:26] "OPTIONS /getSurvey HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 16:25:27] "POST /getSurvey HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 16:25:28] "OPTIONS /getUserSurveys HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 16:25:29] "POST /getUserSurveys HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 16:25:32] "OPTIONS /getSurvey H

{u'surveyName': u'Testing Survey'}


127.0.0.1 - - [19/Oct/2019 16:25:33] "POST /getSurveyEntries HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 16:25:33] "POST /getSurvey HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 16:27:57] "OPTIONS /getSurvey HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 16:27:57] "OPTIONS /getSurvey HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 16:27:57] "OPTIONS /getSurveyEntries HTTP/1.1" 200 -


{u'surveyName': u'Testing Survey'}


...

127.0.0.1 - - [19/Oct/2019 18:45:08] "POST /getSurvey HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 18:45:08] "POST /getSurvey HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 18:45:08] "POST /getSurveyEntries HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 18:46:29] "OPTIONS /getSurvey HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 18:46:30] "OPTIONS /getSurvey HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 18:46:30] "OPTIONS /getSurveyEntries HTTP/1.1" 200 -


{u'surveyName': u'Testing Survey'}


127.0.0.1 - - [19/Oct/2019 18:46:30] "POST /getSurvey HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 18:46:30] "POST /getSurveyEntries HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 18:46:30] "POST /getSurvey HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 18:47:06] "OPTIONS /getSurvey HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 18:47:07] "POST /getSurvey HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 18:47:07] "OPTIONS /getSurvey HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 18:47:07] "OPTIONS /getSurveyEntries HTTP/1.1" 200 -


{u'surveyName': u'Testing Survey'}


127.0.0.1 - - [19/Oct/2019 18:47:08] "POST /getSurvey HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 18:47:08] "POST /getSurveyEntries HTTP/1.1" 200 -
